In [ ]:
# Uncomment the line below to install on Colab or similar
#! pip install git+https://github.com/monash-emu/wpro-working.git@more-datasets

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import numpyro
from numpyro import distributions as dist
from numpyro import infer
from jax import jit, random
import arviz as az
from estival.sampling import tools as esamp
from plotly.express.colors import qualitative as qual_colours

from emu_renewal.outputs import get_spaghetti_from_params, get_quant_df_from_spaghetti, plot_uncertainty_patches, new_new_get_spaghetti
from emu_renewal.process import CosineMultiCurve
from emu_renewal.distributions import GammaDens
from emu_renewal.renew import RenewalModel, ModelResult
from emu_renewal.calibration import StandardCalib
from emu_renewal.targets import StandardTarget

In [ ]:
sars_data = pd.read_csv("https://github.com/monash-emu/wpro-working/raw/more-datasets/data/sars_hongkong/sars_2003_complete_dataset_clean.csv", index_col="Date", parse_dates=True)
# print("Available countries are:")
# set(sars_data["Country"])

In [ ]:
country_req = "Hong Kong SAR, China"
case_data = sars_data[sars_data["Country"] == country_req]["Cumulative number of case(s)"].diff().rolling(4).mean().dropna()

In [ ]:
# Specify fixed parameters and get calibration data
window_len = 14
proc_update_freq = 4
pop = 6.7e6
analysis_start = sars_data.index[0]
analysis_end = datetime(2003, 5, 1)
select_data = case_data.loc[analysis_start: analysis_end]

In [ ]:
# Create exponentially increasing case counts for init_duration period before analysis starts
# Pad with zeroes to allow looking back over the full window
init_duration = 14
exp_coeff = np.log(case_data.iloc[0]) / init_duration
init_series = np.exp(exp_coeff * np.arange(init_duration))

In [ ]:
fitter = CosineMultiCurve()
renew_model = RenewalModel(33e6, analysis_start, analysis_end, proc_update_freq, fitter, GammaDens(), window_len, init_series, GammaDens())

In [ ]:
# Define parameter ranges
priors = {
    "gen_mean": dist.Uniform(6.5, 10.5),
    "gen_sd": dist.Uniform(3.0, 4.6),
    "cdr": dist.Beta(10.0, 4.0),
    "rt_init": dist.Normal(0.0, 0.25),
}
fixed_params = {
    "report_mean": 10.0,
    "report_sd": 5.0
}

In [ ]:
calib_data = {
    "cases": StandardTarget(select_data, 0.1)
}

In [ ]:
calib = StandardCalib(renew_model, priors | fixed_params, calib_data)
kernel = infer.NUTS(calib.calibration, dense_mass=True, init_strategy=calib.custom_init(radius=0.5))
mcmc = infer.MCMC(kernel, num_chains=2, num_samples=1000, num_warmup=1000)
mcmc.run(random.PRNGKey(1))

In [ ]:
idata = az.from_numpyro(mcmc)
idata_sampled = az.extract(idata, num_samples=100)
sample_params = esamp.xarray_to_sampleiterator(idata_sampled)

In [ ]:
spaghetti = new_new_get_spaghetti(calib, sample_params)
key_outputs = ["weekly_sum", "suscept", "r_t", "process"]
quantiles = get_quant_df_from_spaghetti(spaghetti, quantiles=[0.05, 0.5, 0.95])

In [ ]:
plot_uncertainty_patches(quantiles, select_data, qual_colours.Plotly, outputs=key_outputs)

In [ ]:
az.summary(idata)